## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([28 * 28, 256]), name="Weight1")
        self.b1 = tf.Variable(tf.random.normal([1, 256]), name="bias1")
        
        self.W2 = tf.Variable(tf.random.normal([256, 128]), name="Weight2")
        self.b2 = tf.Variable(tf.random.normal([1, 128]), name="bias2")
        
        self.W3 = tf.Variable(tf.random.normal([128, 10]), name="Weight3")
        self.b3 = tf.Variable(tf.random.normal([1, 10]), name="bias3")
        
    @tf.function
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, shape=(-1, 28 * 28))
        
        h1 = tf.nn.relu(x @ self.W1 + self.b1)
        h2 = tf.nn.relu(h1 @ self.W2 + self.b2)
        logits = h2 @ self.W3 + self.b3
        
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [7]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 52.13048 ; accuracy 0.7995
epoch 1 : loss 51.84316 ; accuracy 0.8
epoch 2 : loss 51.55933 ; accuracy 0.8006667
epoch 3 : loss 51.279415 ; accuracy 0.80156666
epoch 4 : loss 51.004433 ; accuracy 0.80216664
epoch 5 : loss 50.734016 ; accuracy 0.80266666
epoch 6 : loss 50.467487 ; accuracy 0.80345
epoch 7 : loss 50.205124 ; accuracy 0.8042167
epoch 8 : loss 49.94667 ; accuracy 0.80478334
epoch 9 : loss 49.69243 ; accuracy 0.8057167
epoch 10 : loss 49.44269 ; accuracy 0.80658334
epoch 11 : loss 49.197193 ; accuracy 0.80726665
epoch 12 : loss 48.955208 ; accuracy 0.8078167
epoch 13 : loss 48.71671 ; accuracy 0.80838335
epoch 14 : loss 48.48131 ; accuracy 0.8088833
epoch 15 : loss 48.24946 ; accuracy 0.8097
epoch 16 : loss 48.0216 ; accuracy 0.8103667
epoch 17 : loss 47.79725 ; accuracy 0.81083333
epoch 18 : loss 47.57604 ; accuracy 0.8113833
epoch 19 : loss 47.35783 ; accuracy 0.81196666
epoch 20 : loss 47.142334 ; accuracy 0.8124
epoch 21 : loss 46.929615 ; accuracy 0.812983